In [9]:
# Importation des bibliothèques nécessaires
from pyspark.sql import SparkSession

In [10]:
# Variables
# Paramètres de connexion MySQL
db_url = "jdbc:mysql://mysql-container:3306/wildlens?serverTimezone=UTC"
db_properties = { "user": "root", "password": "root", "driver": "com.mysql.cj.jdbc.Driver" }

In [11]:
# Initialisation de SparkSession
spark = SparkSession.builder \
    .appName("WildLens ETL - MSPR 24-25") \
    .config("spark.jars", "/installation/mysql-connector-j-9.1.0.jar") \
    .getOrCreate()

25/02/08 15:20:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
# Chargement des informations sur les espèces
species_info_path = './data/csv/infos_especes.csv'
species_info_df = spark.read.csv(species_info_path, sep=";",header=True, inferSchema=True)
species_info_df.show(5)


25/02/08 15:20:09 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
25/02/08 15:20:09 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
25/02/08 15:20:09 INFO FileSourceStrategy: Pushed Filters: 
25/02/08 15:20:09 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#248, None)) > 0)
25/02/08 15:20:09 INFO MemoryStore: Block broadcast_44 stored as values in memory (estimated size 200.3 KiB, free 433.7 MiB)
25/02/08 15:20:09 INFO MemoryStore: Block broadcast_44_piece0 stored as bytes in memory (estimated size 34.6 KiB, free 433.6 MiB)
25/02/08 15:20:09 INFO BlockManagerInfo: Added broadcast_44_piece0 in memory on cedric-galaxy-book.home:33281 (size: 34.6 KiB, free: 434.3 MiB)
25/02/08 15:20:09 INFO SparkContext: Created broadcast 44 from csv at NativeMethodAccessorImpl.java:0
25/02/08 15:20:09 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
25/02/08 15:20:0

+--------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|  Espèce|         Description|           Nom latin|  Famille|              Taille|              Région|             Habitat|            Fun fact|
+--------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|  Castor|Le castor d’Europ...|   Castor canadensis|Mammifère|100 à 135 cm queu...|Europe du nord et...|Le castor d’Europ...|À l’exception des...|
|    Chat|Le Chat sauvage4 ...|    Felis silvestris|Mammifère|46 à 51 cm sans l...|Amérique, Europe,...|                NULL|Un chat ne peut p...|
|   Chien|Alors qu'on estim...|Canis lupus famil...|Mammifère|                NULL|     Hémisphère nord|                NULL|L'empreinte de la...|
|  Coyote|Le coyote, égalem...|       Canis latrans|Mammifère|         70 à 120 cm|Amérique du nord,...|Le coyote vit 

25/02/08 15:20:09 INFO BlockManagerInfo: Removed broadcast_47_piece0 on cedric-galaxy-book.home:33281 in memory (size: 12.9 KiB, free: 434.3 MiB)


In [13]:
# Remplacer les valeurs manquantes par "valeur manquante"
df_filled = species_info_df.fillna("missing value")

# Identifier les colonnes sans nom
columns_with_name = [col for col in df_filled.columns if col.strip()]

# Identifier les colonnes aberrantes (avec une seule valeur unique)
abnormal_columns = []
for col_name in columns_with_name:
    unique_value_count = df_filled.select(col_name).distinct().count()
    if unique_value_count <= 1:
        abnormal_columns.append(col_name)

# Supprimer les colonnes aberrantes
columns_to_keep = [col for col in columns_with_name if col not in abnormal_columns]
cleaned_df = df_filled.select(*columns_to_keep)

# Afficher un aperçu
cleaned_df.show()


25/02/08 15:20:09 INFO FileSourceStrategy: Pushed Filters: 
25/02/08 15:20:09 INFO FileSourceStrategy: Post-Scan Filters: 
25/02/08 15:20:09 INFO MemoryStore: Block broadcast_50 stored as values in memory (estimated size 200.2 KiB, free 433.9 MiB)
25/02/08 15:20:09 INFO MemoryStore: Block broadcast_50_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.9 MiB)
25/02/08 15:20:09 INFO BlockManagerInfo: Added broadcast_50_piece0 in memory on cedric-galaxy-book.home:33281 (size: 34.5 KiB, free: 434.3 MiB)
25/02/08 15:20:09 INFO SparkContext: Created broadcast 50 from count at NativeMethodAccessorImpl.java:0
25/02/08 15:20:09 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
25/02/08 15:20:09 INFO DAGScheduler: Registering RDD 123 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 16
25/02/08 15:20:09 INFO DAGScheduler: Got map stage job 33 (count at NativeMethodAccessorImpl.java:0)

+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Espèce|         Description|           Nom latin|              Taille|              Région|             Habitat|            Fun fact|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Castor|Le castor d’Europ...|   Castor canadensis|100 à 135 cm queu...|Europe du nord et...|Le castor d’Europ...|À l’exception des...|
|        Chat|Le Chat sauvage4 ...|    Felis silvestris|46 à 51 cm sans l...|Amérique, Europe,...|       missing value|Un chat ne peut p...|
|       Chien|Alors qu'on estim...|Canis lupus famil...|       missing value|     Hémisphère nord|       missing value|L'empreinte de la...|
|      Coyote|Le coyote, égalem...|       Canis latrans|         70 à 120 cm|Amérique du nord,...|Le coyote vit dan...|Animal social et ...|
|    Ecureuil

25/02/08 15:20:10 INFO MemoryStore: Block broadcast_82_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.5 MiB)
25/02/08 15:20:10 INFO BlockManagerInfo: Added broadcast_82_piece0 in memory on cedric-galaxy-book.home:33281 (size: 34.5 KiB, free: 434.2 MiB)
25/02/08 15:20:10 INFO SparkContext: Created broadcast 82 from showString at NativeMethodAccessorImpl.java:0
25/02/08 15:20:10 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
25/02/08 15:20:10 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
25/02/08 15:20:10 INFO DAGScheduler: Got job 57 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
25/02/08 15:20:10 INFO DAGScheduler: Final stage: ResultStage 105 (showString at NativeMethodAccessorImpl.java:0)
25/02/08 15:20:10 INFO DAGScheduler: Parents of final stage: List()
25/02/08 15:20:10 INFO DAGScheduler: Missing parents: List()
25/02

In [14]:
# Sauvegarde au format CSV
cleaned_df.write.csv("./output/data/nettoye.csv", header=True, mode="overwrite")

# Sauvegarde au format Parquet
cleaned_df.write.parquet("./output/data/nettoye.parquet", mode="overwrite")


25/02/08 15:20:10 INFO FileSourceStrategy: Pushed Filters: 
25/02/08 15:20:10 INFO FileSourceStrategy: Post-Scan Filters: 
25/02/08 15:20:10 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
25/02/08 15:20:10 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
25/02/08 15:20:10 INFO SQLHadoopMapReduceCommitProtocol: Using output committer class org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter
25/02/08 15:20:10 INFO MemoryStore: Block broadcast_84 stored as values in memory (estimated size 200.2 KiB, free 433.9 MiB)
25/02/08 15:20:10 INFO MemoryStore: Block broadcast_84_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.9 MiB)
25/02/08 15:20:10 INFO BlockManagerInfo: Added broadcast_84_piece0 in memory on cedric-galaxy-book.home:33281 (size: 34.5 KiB, free: 434.3 MiB)
25/02/08 15:20:10 INFO SparkContext: Created broadcast 84 from csv at NativeMethodAcces

In [15]:
# Écrire les données dans la table MySQL
cleaned_df.write \
    .jdbc(url=db_url, table="Animaux", mode="overwrite", properties=db_properties)


Py4JJavaError: An error occurred while calling o226.jdbc.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:258)
	at